In [ ]:
ENV["Code"] = "../../Code"
for folder in readdir(ENV["Code"]); push!(LOAD_PATH, normpath(ENV["Code"], folder));end

using Seaborn, PyPlot, PyCall, DataFrames
using NoLongerProblems_Pandas, Pandas
using MultipleTesting, HypothesisTests 
using SingleCellExperiment, StatsBase
using ProgressMeter, Random
using InformationMeasures
using CSV, ProgressMeter,PrettyPlotting, ScikitLearn
@sk_import linear_model: LogisticRegression

include("../Databases/Cuartero2018.jl")
include("../Databases/Bhatt2012.jl")
include("../Databases/MF_SingleCell.jl")
include("../Code/Load_scRNAseqData.jl")

function calculate_entropy_per_cell(sceexp, assay)
    expmatrix = sceexp.assays[assay]
    ncells = size(expmatrix)[2]
    sceexp.colData[!, string(assay*"_Entropy")] = [get_entropy(expmatrix[:,ii]; base = 2, number_of_bins = 10) for ii in 1:ncells]
    sceexp
end

function calculate_mutualinformation(sceexp, assay)
    expmatrix = sceexp.assays[assay]
    ncells = size(expmatrix)[2]
    newassay = zeros(ncells, ncells)
    
    for ii in 1:ncells
        for jj in 1:ncells
            if ii != jj && newassay[ii, jj] == 0.0 
                mi  = get_mutual_information(expmatrix[:,ii], expmatrix[:,jj], base = 2, number_of_bins = 10)
                newassay[ii, jj] = mi
                newassay[jj, ii] = mi
            end
        end
    end
    sceexp.assays[string(assay, "_MI")] = newassay
    sceexp
end

function calculate_normalised_mutual_information(sceexp, assay)
    h = sceexp.colData[!,assay*"_Entropy"]
    mi = sceexp.assays[assay*"_MI"]
    expmatrix = sceexp.assays[assay]
    ncells = size(expmatrix)[2]
    newassay = zeros(ncells, ncells)
    for ii in 1:ncells
        for jj in (ii+1):ncells
            if ii != jj && newassay[ii, jj] == 0.0 
                nmi = mi[ii, jj] / sqrt(h[ii]*h[jj])
                newassay[ii, jj] = nmi
                newassay[jj, ii] = nmi
            end
        end
    end
    sceexp.assays[string(assay, "_NMI")] = newassay
    sceexp
end


sce = SingleCellExperiment.get_cells_with_this_characteristics(["WT", "RAD21"], :Genotype, sce)
sce = SingleCellExperiment.get_cells_with_this_characteristics(["UT"], :Timepoint, sce)
sce = SingleCellExperiment.select_expressed_genes(sce; min_cells_expressing_gene = 100)
sce = SingleCellExperiment.fit_mu_std_alpha(sce, splitdataby = :Sample, assay = "CPM")
bhattgenes = Bhatt2012.inducible_genes_figure3()[!,:GeneSymbol];
sce = calculate_entropy_per_cell(sce, "lnCPMplus1")
sce = calculate_entropy_per_cell(sce, "CPM")
sce = calculate_mutualinformation(sce, "lnCPMplus1")
sce = calculate_mutualinformation(sce, "CPM")
sce = calculate_normalised_mutual_information(sce, "lnCPMplus1")
sce = calculate_normalised_mutual_information(sce, "CPM")

In [ ]:

function make_nmi_figure(sceexp, assay; times= 100, cellssubsampled = 120)
wt = SingleCellExperiment.get_cells_with_this_characteristics(["WT"], :Genotype, sceexp)
rad = SingleCellExperiment.get_cells_with_this_characteristics(["RAD21"], :Genotype, sceexp)
# we subsampled 100 cells from each time-point (or mouse) 100 times and calculated the median NMI across each within-timepoint sampled pair.  
nmi_wt = zeros(times)
nmi_rad = zeros(times)
nwt = size(wt.assays[assay])[2]
nrad = size(rad.assays[assay])[2]

for aa in 1:times
    wt_cols = shuffle(1:nwt)[1:cellssubsampled]
    rad_cols = shuffle(1:nrad)[1:cellssubsampled]
    nmis_wt = []
    nmis_rad = []
        
        for ii in 1:cellssubsampled
            for jj in (ii+1):cellssubsampled
                iiw = wt_cols[ii]
                jjw = wt_cols[jj]
                iir = rad_cols[ii]
                jjr = rad_cols[jj]
                push!(nmis_wt, wt.assays[assay*"_NMI"][iiw,jjw])
                push!(nmis_rad, rad.assays[assay*"_NMI"][iir,jjr])
            end
        end
    nmi_wt[aa] =  StatsBase.median(nmis_wt)
    nmi_rad[aa] =  StatsBase.median(nmis_rad)
end
    

    plt.boxplot([nmi_wt, nmi_rad])
    plt.xticks(1:2,["WT", "Rad21KO"])
    
    abiggerb = zeros(times, times)
    
    for ii in 1:times 
        for jj in 1:times
        abiggerb[ii, jj] = (nmi_wt[ii] > nmi_rad[jj]) + 1
        end
    end
    
    println(MannWhitneyUTest(nmi_wt,nmi_rad))
    
    p = (sum(nmi_wt .< nmi_rad)+1)/(times+1)
    println("P as calculated as (r+1)/(n+1) = $p")
    ylabel("NMI")
end

make_nmi_figure(sce, "CPM")



